# Assignment03. Exploring Urban Mobility with Spatial Data

## Load the Chicago Transportation Network Providers Data as a JSON API

In [1]:
!pip install fiona
!pip install geopandas
!pip install sodapy -q
!pip install adjustText
!pip install plotly --upgrade
!pip install mapclassify
!pip install chart-studio

     |████████████████████████████████| 15.4 MB 35 kB/s 
     |████████████████████████████████| 1.0 MB 5.5 MB/s 
     |████████████████████████████████| 6.3 MB 31.9 MB/s 
  Created wheel for adjustText: filename=adjustText-0.7.3-py3-none-any.whl size=7094 sha256=096a296ff0d1b23906ab6eb89eb658f86a4f481001fdf197827fc97425881c57
  Stored in directory: /root/.cache/pip/wheels/2f/98/32/afbf902d8f040fadfdf0a44357e4ab750afe165d873bf5893d
Successfully built adjustText
     |████████████████████████████████| 25.3 MB 53 kB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 64 kB 1.8 MB/s 


In [22]:
import os
import fiona
from sodapy import Socrata
import pandas as pd
from IPython.display import Image, display
from shapely.geometry import Polygon
import time
from shapely.geometry import shape

In [25]:
!git clone https://github.com/kirthi-b/Chicago-Rideshare

Cloning into 'Chicago-Rideshare'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4 (delta 0), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.


In [ ]:
gdf = gpd.read_file("/content/NoiseNYC/ZIP_CODE_040114", driver = "shapefile")
gdf.sort_values('ZIPCODE',inplace=True, ascending=True)
gdf.head()

In [5]:
client = Socrata("data.cityofchicago.org", None)
results = client.get("igwz-8jzy", limit=2000)
ca_df = pd.DataFrame.from_records(results)

In [6]:
ca_df.head()

,the_geom,perimeter,area,comarea,comarea_id,area_numbe,community,area_num_1,shape_area,shape_len
0,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",0,0,0,0,35,DOUGLAS,35,46004621.1581,31027.0545098
1,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",0,0,0,0,36,OAKLAND,36,16913961.0408,19565.5061533
2,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",0,0,0,0,37,FULLER PARK,37,19916704.8692,25339.0897503
3,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",0,0,0,0,38,GRAND BOULEVARD,38,48492503.1554,28196.8371573
4,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",0,0,0,0,39,KENWOOD,39,29071741.9283,23325.1679062


In [20]:
split_ca_df = pd.DataFrame(ca_df['the_geom'].tolist())

In [21]:
split_ca_df.head()

,type,coordinates
0,MultiPolygon,"[[[[-87.60914087617894, 41.84469250265398], [-..."
1,MultiPolygon,"[[[[-87.59215283879394, 41.81692934626684], [-..."
2,MultiPolygon,"[[[[-87.62879823733725, 41.80189303368919], [-..."
3,MultiPolygon,"[[[[-87.6067081256125, 41.81681377057218], [-8..."
4,MultiPolygon,"[[[[-87.59215283879394, 41.81692934626684], [-..."


In [24]:
split_ca_df['coordinates'] = split_ca_df['coordinates'].apply(Polygon)
split_ca_df.head()

ValueError: ignored

In [23]:
 # MultiPolygon from the list of Polygons
 Multi = MultiPolygon([shape(pol['the_geom']) for pol in fiona.open('poly.shp')])
 Multi.wkt

NameError: ignored